# Data Pre-processing

First we will begin by pre-processing the data. In this case the sample data comes with the following features/columns: 

|   **Column**   |   **Desciption**   |
|:-:	         |:--	              |
|   STUD	|   Study Number 	|
|   DSFQ	|   Dosing Frequency	|
|   PTNM	|   Patient Number	|
|   CYCL	|    Dosing Cycles	|
|   AMT	|   Dosing Amounts	|
|   TIME	|   Time in Hours Since the Experiment Began for one Individual	|
|   TFDS	|   Time in Hours Since the Last Dosing|
|   DV/PK_timeCourse	|   The Observations of PK  	|

Let us first import the data into the notebook and observe these features of the dataset

In [5]:
# Importing required libraries for data pre-processing

import pandas as pd
import numpy as np

#Reading the csv file with the data

data_complete = pd.read_csv("/Users/rishabhgoel/Desktop/NeuralODE_Paper_Supplementary_Code/ExampleData/sim_data.csv", na_values='.')

data_complete

,STUD,PTNM,DSFQ,CYCL,AMT,TIME,TFDS,DV
0,1000.0,1.0,3.0,1.0,296.6,0.0,0.0,20.382000
1,1000.0,1.0,3.0,1.0,0.0,24.0,24.0,73.148000
2,1000.0,1.0,3.0,1.0,0.0,216.0,216.0,19.764000
3,1000.0,1.0,3.0,1.0,0.0,504.0,504.0,3.219900
4,1000.0,1.0,3.0,2.0,288.0,504.0,0.0,3.219900
...,...,...,...,...,...,...,...,...
5371,3000.0,200.0,3.0,15.0,0.0,7416.0,360.0,6.091200
5372,3000.0,200.0,3.0,15.0,0.0,7584.0,7584.0,2.043100
5373,3000.0,200.0,3.0,17.0,259.2,8064.0,0.0,0.090115
5374,3000.0,200.0,3.0,17.0,0.0,8088.0,24.0,53.990000


There are 5376 observations and 8 features that we are looking at. Now let us begin the pre-processing the data. 

Our sample data only has the relevant columns for the model. However, in the data we receive from patients there will be many more features so it is important for us to select the correct features. Thus, let us begin by creating a place holder for the features that are important to the model.

In [6]:
#variable for colummns that we will eventually select from the raw dataset we receive

select_cols = ["STUD", "DSFQ", "PTNM", "CYCL", "AMT", "TIME", "TFDS", "DV"]

#Selecting the relevant columns from the dataframe

data_complete = data_complete[select_cols]

We then start filtering data based on multiple parameters:
    
1. Dosing Cycle < 100

In [7]:
# filtering for only the rows in the dataframe with a dosing cycle less than 100

data_complete = data_complete[data_complete.CYCL < 100]
data_complete

,STUD,PTNM,DSFQ,CYCL,AMT,TIME,TFDS,DV
0,1000.0,1.0,3.0,1.0,296.6,0.0,0.0,20.382000
1,1000.0,1.0,3.0,1.0,0.0,24.0,24.0,73.148000
2,1000.0,1.0,3.0,1.0,0.0,216.0,216.0,19.764000
3,1000.0,1.0,3.0,1.0,0.0,504.0,504.0,3.219900
4,1000.0,1.0,3.0,2.0,288.0,504.0,0.0,3.219900
...,...,...,...,...,...,...,...,...
5371,3000.0,200.0,3.0,15.0,0.0,7416.0,360.0,6.091200
5372,3000.0,200.0,3.0,15.0,0.0,7584.0,7584.0,2.043100
5373,3000.0,200.0,3.0,17.0,259.2,8064.0,0.0,0.090115
5374,3000.0,200.0,3.0,17.0,0.0,8088.0,24.0,53.990000


We then convert the "PTNM" column to an integer type using the astype() method, and then using the map() method to apply a format string to each value in the column. The format string "{:05d}" specifies that each value should be formatted as a zero-padded integer with a width of 5 digits.

For example, if the "PTNM" column originally contained the values [1, 10, 100, 1000], after this line of code is executed, the "PTNM" column would contain the values ['00001', '00010', '00100', '01000'].

This line of code is useful for standardizing the format of the values in the "PTNM" column, which can make it easier to perform operations on the column and to compare values within the column. It can also be useful for preparing the data for downstream model applications that require the data to be in a particular format.

In [9]:
# formatting Patient Number for Unique Identifier
data_complete["PTNM"] = data_complete["PTNM"].astype("int").map("{:05d}".format)
data_complete

,STUD,PTNM,DSFQ,CYCL,AMT,TIME,TFDS,PK_timeCourse,ID
0,1000.0,00001,3.0,1.0,296.6,0.0,0.0,20.382000,100000001
1,1000.0,00001,3.0,1.0,0.0,24.0,24.0,73.148000,100000001
2,1000.0,00001,3.0,1.0,0.0,216.0,216.0,19.764000,100000001
3,1000.0,00001,3.0,1.0,0.0,504.0,504.0,3.219900,100000001
4,1000.0,00001,3.0,2.0,288.0,504.0,0.0,3.219900,100000001
...,...,...,...,...,...,...,...,...,...
5371,3000.0,00200,3.0,15.0,0.0,7416.0,360.0,6.091200,300000200
5372,3000.0,00200,3.0,15.0,0.0,7584.0,7584.0,2.043100,300000200
5373,3000.0,00200,3.0,17.0,259.2,8064.0,0.0,0.090115,300000200
5374,3000.0,00200,3.0,17.0,0.0,8088.0,24.0,53.990000,300000200


Now we create a new column called "ID" in the pandas DataFrame data_complete. The "ID" column is being created by concatenating two existing columns in data_complete: "STUD" and "PTNM".

First, the "STUD" column is being converted to an integer data type using the astype() method with the argument "int". Then, the "STUD" column is being converted to a string data type using the astype() method with the argument "str". This ensures that the values in the "STUD" column are in string format.

Next, the values in the "STUD" and "PTNM" columns are being concatenated using the + operator, which joins the two strings together. This creates a new string for each row in the DataFrame, which is assigned to the "ID" column.

For example, if the "STUD" column contained the values ['100', '101', '102', '103'] and the "PTNM" column contained the values ['00001', '00010', '00100', '01000'], then after this line of code is executed, the "ID" column would contain the values ['10000001', '10100010', '10200100', '10301000'].

This line of code is useful for creating a unique identifier for each row in the DataFrame based on the values in the "STUD" and "PTNM" columns. This can be useful for identifying and tracking individual records, as well as for linking data across multiple datasets.

In [11]:
# Creating a unique identifier column, 'ID', and formatting it based on Patient Number and Study Number
data_complete["ID"] = data_complete["STUD"].astype("int").astype("str") + data_complete["PTNM"]
data_complete

,STUD,PTNM,DSFQ,CYCL,AMT,TIME,TFDS,PK_timeCourse,ID
0,1000.0,00001,3.0,1.0,296.6,0.0,0.0,20.382000,100000001
1,1000.0,00001,3.0,1.0,0.0,24.0,24.0,73.148000,100000001
2,1000.0,00001,3.0,1.0,0.0,216.0,216.0,19.764000,100000001
3,1000.0,00001,3.0,1.0,0.0,504.0,504.0,3.219900,100000001
4,1000.0,00001,3.0,2.0,288.0,504.0,0.0,3.219900,100000001
...,...,...,...,...,...,...,...,...,...
5371,3000.0,00200,3.0,15.0,0.0,7416.0,360.0,6.091200,300000200
5372,3000.0,00200,3.0,15.0,0.0,7584.0,7584.0,2.043100,300000200
5373,3000.0,00200,3.0,17.0,259.2,8064.0,0.0,0.090115,300000200
5374,3000.0,00200,3.0,17.0,0.0,8088.0,24.0,53.990000,300000200


In [15]:
# Creating a dataframe with the maximum time for each patient in each study (Essentially presenting the time of the last observed dose)
time_summary = data_complete[["ID", "TIME"]].groupby("ID").max().reset_index()

# Creating a new dataframe with only the IDs for patients with a time of the last observed dose greater than 0 (eliminating errors/outliers)
selected_ptnms = time_summary[time_summary.TIME > 0].ID

# Only selecting IDs with the last observed dose greater than 0 in our main table called data_complete
data_complete = data_complete[data_complete.ID.isin(selected_ptnms)]
data_complete

,STUD,PTNM,DSFQ,CYCL,AMT,TIME,TFDS,PK_timeCourse,ID
0,1000.0,00001,3.0,1.0,296.6,0.0,0.0,20.382000,100000001
1,1000.0,00001,3.0,1.0,0.0,24.0,24.0,73.148000,100000001
2,1000.0,00001,3.0,1.0,0.0,216.0,216.0,19.764000,100000001
3,1000.0,00001,3.0,1.0,0.0,504.0,504.0,3.219900,100000001
4,1000.0,00001,3.0,2.0,288.0,504.0,0.0,3.219900,100000001
...,...,...,...,...,...,...,...,...,...
5371,3000.0,00200,3.0,15.0,0.0,7416.0,360.0,6.091200,300000200
5372,3000.0,00200,3.0,15.0,0.0,7584.0,7584.0,2.043100,300000200
5373,3000.0,00200,3.0,17.0,259.2,8064.0,0.0,0.090115,300000200
5374,3000.0,00200,3.0,17.0,0.0,8088.0,24.0,53.990000,300000200


In [17]:
# filling in missing values in the "AMT" column with Dosing Amounts equal to 0
data_complete["AMT"] = data_complete["AMT"].fillna(0)


#Renaming the column DV to PK_timeCourse so that the name is more self-explanatory
data_complete = data_complete.rename(columns={"DV": "PK_timeCourse"})


# Duplicating PK_timeCourse column to make changes to it separately
data_complete["PK_round1"] = data_complete["PK_timeCourse"]

# Changing the PK_round1 such that if the dosing frequency is once weekly and the time of the last 
# observed dose is greater than 168hrs the PK for round 1 is zero. Similarly, if the dosing frequency 
# is once weekly and the time of the last observed dose is greater than 504hrs the PK for round 1 is zero
data_complete.loc[(data_complete.DSFQ == 1) & (data_complete.TIME >= 168), "PK_round1"] = 0
data_complete.loc[(data_complete.DSFQ == 3) & (data_complete.TIME >= 504), "PK_round1"] = 0

# filling in missing values in the "PK_round1" column with 0
data_complete["PK_round1"] = data_complete["PK_round1"].fillna(0)

# filling in missing values in the "PK_timeCourse" column with -1
data_complete["PK_timeCourse"] = data_complete["PK_timeCourse"].fillna(-1)
data_complete

,STUD,PTNM,DSFQ,CYCL,AMT,TIME,TFDS,PK_timeCourse,ID,PK_round1
0,1000.0,00001,3.0,1.0,296.6,0.0,0.0,20.382000,100000001,20.382
1,1000.0,00001,3.0,1.0,0.0,24.0,24.0,73.148000,100000001,73.148
2,1000.0,00001,3.0,1.0,0.0,216.0,216.0,19.764000,100000001,19.764
3,1000.0,00001,3.0,1.0,0.0,504.0,504.0,3.219900,100000001,0.000
4,1000.0,00001,3.0,2.0,288.0,504.0,0.0,3.219900,100000001,0.000
...,...,...,...,...,...,...,...,...,...,...
5371,3000.0,00200,3.0,15.0,0.0,7416.0,360.0,6.091200,300000200,0.000
5372,3000.0,00200,3.0,15.0,0.0,7584.0,7584.0,2.043100,300000200,0.000
5373,3000.0,00200,3.0,17.0,259.2,8064.0,0.0,0.090115,300000200,0.000
5374,3000.0,00200,3.0,17.0,0.0,8088.0,24.0,53.990000,300000200,0.000


In [20]:
# Removing rows where the "AMT" column is 0 and the "TIME" column is also 0.
data_complete = data_complete[~((data_complete.AMT == 0) & (data_complete.TIME == 0))]


# Keeping the last row for all patients with duplicate values for time in hours 
data_complete.loc[data_complete[["PTNM", "TIME"]].duplicated(keep="last"), "AMT"] = \
    data_complete.loc[data_complete[["PTNM", "TIME"]].duplicated(keep="first"), "AMT"].values

# Keeping the first row for all observations with duplicate values for all features apart from patient and time in hours 
data_complete = data_complete[~data_complete[["PTNM", "TIME"]].duplicated(keep="first")]

data_complete

,STUD,PTNM,DSFQ,CYCL,AMT,TIME,TFDS,PK_timeCourse,ID,PK_round1
0,1000.0,00001,3.0,1.0,296.6,0.0,0.0,20.382000,100000001,20.382
1,1000.0,00001,3.0,1.0,0.0,24.0,24.0,73.148000,100000001,73.148
2,1000.0,00001,3.0,1.0,0.0,216.0,216.0,19.764000,100000001,19.764
3,1000.0,00001,3.0,1.0,288.0,504.0,504.0,3.219900,100000001,0.000
5,1000.0,00001,3.0,2.0,0.0,696.0,192.0,23.210000,100000001,0.000
...,...,...,...,...,...,...,...,...,...,...
5371,3000.0,00200,3.0,15.0,0.0,7416.0,360.0,6.091200,300000200,0.000
5372,3000.0,00200,3.0,15.0,0.0,7584.0,7584.0,2.043100,300000200,0.000
5373,3000.0,00200,3.0,17.0,259.2,8064.0,0.0,0.090115,300000200,0.000
5374,3000.0,00200,3.0,17.0,0.0,8088.0,24.0,53.990000,300000200,0.000
